In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-dec-2021/train.csv
/kaggle/input/tabular-playground-series-dec-2021/test.csv


In [3]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv')

In [4]:
pd.options.display.max_columns = train.shape[1]
train.describe(include = 'all').apply(lambda s: s.apply('{0:.5f}'.format))

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000,4000000.00000
mean,1999999.50000,2980.19166,151.58567,15.09754,271.31544,51.66262,1766.64163,211.83754,221.06144,140.81087,1581.40681,0.26119,0.04166,0.65357,0.02182,0.01684,0.03090,0.00428,0.03791,0.01572,0.00797,0.00000,0.00290,0.01089,0.05454,0.02799,0.01829,0.03130,0.01498,0.00000,0.01589,0.02067,0.01344,0.01381,0.01737,0.01154,0.03135,0.04917,0.02502,0.00326,0.01353,0.01177,0.01071,0.02227,0.02887,0.02749,0.03746,0.03782,0.01200,0.01605,0.01071,0.01221,0.04075,0.03924,0.03162,1.77134
std,1154700.68272,289.04817,109.96112,8.54673,226.54968,68.21597,1315.60989,30.75996,22.23134,43.69864,1127.61617,0.43928,0.19981,0.47583,0.14609,0.12868,0.17304,0.06525,0.19099,0.12437,0.08893,0.00000,0.05377,0.10380,0.22708,0.16493,0.13400,0.17411,0.12146,0.00000,0.12504,0.14228,0.11513,0.11671,0.13064,0.10680,0.17425,0.21622,0.15619,0.05699,0.11552,0.10783,0.10292,0.14757,0.16743,0.16352,0.18989,0.19076,0.10886,0.12568,0.10295,0.10981,0.19771,0.19416,0.17498,0.89381
min,0.00000,1773.00000,-33.00000,-3.00000,-92.00000,-317.00000,-287.00000,-4.00000,49.00000,-53.00000,-277.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
25%,999999.75000,2760.00000,60.00000,9.00000,110.00000,4.00000,822.00000,198.00000,210.00000,115.00000,781.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
50%,1999999.50000,2966.00000,123.00000,14.00000,213.00000,31.00000,1436.00000,218.00000,224.00000,142.00000,1361.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,2.00000
75%,2999999.25000,3217.00000,247.00000,20.00000,361.00000,78.00000,2365.00000,23

# Dropping unncecary columns

In [5]:
train.drop('Soil_Type7', axis=1, inplace=True, errors='ignore')
train.drop('Soil_Type15', axis=1, inplace=True, errors='ignore')
train.drop('Id', axis=1, inplace=True, errors='ignore')
test.drop('Soil_Type7',axis=1, inplace=True, errors='ignore')
test.drop('Soil_Type15', axis=1, inplace=True, errors='ignore')
test_Id = test.Id
test.drop('Id', axis=1, inplace=True, errors='ignore')

# Diving target variable

In [6]:
Y = pd.get_dummies(train.Cover_Type)
X = train.drop(['Cover_Type'], axis=1)

# Converting to numpy array

In [7]:
X = X.to_numpy()
Y = Y.to_numpy()

# Train_test_split data

In [8]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=10)

# Random Forest

In [9]:
from sklearn.ensemble import RandomForestRegressor
import sklearn
RF = RandomForestRegressor(n_estimators = 10, random_state = 0, n_jobs=-1, verbose=2) # max_depth, min_samples_split, 
RF.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 10building tree 2 of 10
building tree 3 of 10

building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  5.3min finished


RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=0, verbose=2)

In [10]:
predicted = RF.predict(X_val)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.7s finished


# Decoding one-hot-encoder

In [11]:
for i in range(len(predicted)):
    mx = max(predicted[i])
    for j in range(len(predicted[i])):
        if predicted[i][j] == mx:
            predicted[i][j] = 1
        else:
            predicted[i][j] = 0

# Accuracy

In [12]:
count = 0
for i in range(len(predicted)):
    if not (predicted[i] == Y_val[i]).all():
        count += 1
print(f"Validation Accuracy: {1 - (count/len(predicted))}%")

Validation Accuracy: 0.94758875%


# Prediction test-file

In [13]:
file_prediction = np.zeros([len(test_Id), 1], dtype=int)
test_prediction = RF.predict(test)

for i in range(len(test_prediction)):
    file_prediction[i] = np.argmax(test_prediction[i], axis=0) + 1

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    1.0s finished


# Generating output file

In [14]:
output = pd.DataFrame({'Id': test_Id, 'Cover_Type': file_prediction.flatten()})
output.to_csv('submission.csv', index=False)